<a href="https://colab.research.google.com/github/bonosa/AI-ambients/blob/main/humanoid_wave_only_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# Setup Environment
# ==============================================================================
print("Starting full environment setup...")
!git clone https://github.com/google-deepmind/mujoco_menagerie.git > /dev/null 2>&1
!apt-get install -y --no-install-recommends libgl1-mesa-glx libosmesa6-dev libglew2.2 > /dev/null 2>&1
!apt-get install -y patchelf > /dev/null 2>&1
!apt-get install -y xvfb > /dev/null 2>&1

import os
os.system("Xvfb :99 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':99'

!pip install mujoco imageio-ffmpeg moviepy > /dev/null 2>&1
print("Setup complete.")


Starting full environment setup...
Setup complete.


In [2]:
# ==============================================================================
# Cell 2 – Load MuJoCo model and set up renderer
# ==============================================================================
import mujoco
import numpy as np
import imageio
from google.colab import files

print("Loading Unitree H1 model …")
model_path = "mujoco_menagerie/unitree_h1/h1.xml"
model  = mujoco.MjModel.from_xml_path(model_path)
data   = mujoco.MjData(model)

# ── off-screen renderer ───────────────────────────────────────────────
renderer = mujoco.Renderer(model, height=480, width=640)
renderer.update_scene(data)             # initialise internal Scene

# handle all MuJoCo versions (scene / scn / _scene)
_scene = (getattr(renderer, "scene", None)
          or getattr(renderer, "scn",   None)
          or getattr(renderer, "_scene", None))

# lock exposure & disable auto-exposure if available
if _scene is not None:
    if hasattr(_scene, "exposure"):
        _scene.exposure = 2.0           # constant brightness
    if hasattr(_scene, "flags") and hasattr(mujoco.mjtRndFlag, "mjRND_AUTOMATIC"):
        _scene.flags &= ~mujoco.mjtRndFlag.mjRND_AUTOMATIC

print("Model + renderer ready.")


Loading Unitree H1 model …
Model + renderer ready.


In [3]:
# ==============================================================================
# Cell 3 – Scene configuration & lighting
# ==============================================================================
print("Configuring scene …")

# keep robot hovering
model.opt.gravity[:] = 0
mujoco.mj_forward(model, data)

# camera & render options
scene_option = mujoco.MjvOption()
scene_option.geomgroup[1] = 0          # hide collision geoms
scene_option.geomgroup[0] = 1          # show visual geoms

camera = mujoco.MjvCamera()
mujoco.mjv_defaultCamera(camera)
camera.azimuth, camera.elevation = 135, -15
camera.distance = 3.5
camera.lookat[:] = [0, 0, 0.9]

# ── blast every light ──────────────────────────────────────────
for i in range(model.nlight):
    model.light_ambient[i]  = [2, 2, 2]
    model.light_diffuse[i]  = [2, 2, 2]
    model.light_specular[i] = [2, 2, 2]
    if hasattr(model, "light_attenuation"):
        model.light_attenuation[i] = [1, 0, 0]   # no fall-off

# turn light 0 into a high, directional “sun”
if hasattr(model, "light_directional"):
    model.light_directional[0] = 1               # 1 = directional
    model.light_pos[0] = [0, 0, 10]              # ignored for directional
    model.light_dir[0] = [0, 0, -1]              # shines straight down
    if hasattr(model, "light_castshadow"):
        model.light_castshadow[0] = 0            # disable shadows

print("Scene configured.")


Configuring scene …
Scene configured.


In [4]:
# ==============================================================================
# Get Actuator IDs
# ==============================================================================
print("Getting motor IDs for the wave...")
left_shoulder_pitch_id = model.actuator('left_shoulder_pitch').id
right_shoulder_pitch_id = model.actuator('right_shoulder_pitch').id
print("Motor IDs acquired.")


Getting motor IDs for the wave...
Motor IDs acquired.


In [5]:
# ==============================================================================
# Cell 5 – Generate marching + waving animation
# ==============================================================================
duration  = 5            # seconds
fps       = 30
n_frames  = duration * fps
speed     = 0.4          # m/s forward
frames    = []

# joint indices
l_sh_pitch = model.joint('left_shoulder_pitch').qposadr
r_sh_pitch = model.joint('right_shoulder_pitch').qposadr
l_hip      = model.joint('left_hip_pitch').qposadr
r_hip      = model.joint('right_hip_pitch').qposadr
l_knee     = model.joint('left_knee').qposadr
r_knee     = model.joint('right_knee').qposadr

root_x = 0  # qpos[0]

for k in range(n_frames):
    t   = k / fps
    phi = 2 * np.pi * 1.2 * t        # 1.2 Hz march cadence

    # legs
    hip_amp, knee_amp = 0.5, 0.7
    data.qpos[l_hip]  =  hip_amp  * np.sin(phi)
    data.qpos[r_hip]  = -hip_amp  * np.sin(phi)
    data.qpos[l_knee] =  knee_amp * np.sin(phi)
    data.qpos[r_knee] = -knee_amp * np.sin(phi)

    # arms waving
    wave = 0.6 * np.sin(2 * np.pi * 1.0 * t)
    data.qpos[l_sh_pitch] =  wave
    data.qpos[r_sh_pitch] = -wave

    # slide root forward
    root_x += speed / fps
    data.qpos[0] = root_x

    mujoco.mj_forward(model, data)
    renderer.update_scene(data, camera=camera, scene_option=scene_option)

    # per-frame exposure clamp (stops auto-dimming)
    if _scene is not None:
        if hasattr(_scene, "exposure"):
            _scene.exposure = 2.0
        if (hasattr(_scene, "flags") and
            hasattr(mujoco.mjtRndFlag, "mjRND_AUTOMATIC")):
            _scene.flags &= ~mujoco.mjtRndFlag.mjRND_AUTOMATIC

    frames.append(renderer.render())

print(f"{n_frames} frames generated.")


150 frames generated.


In [ ]:
# ==============================================================================
# Export Video
# ==============================================================================
output_filename_mp4 = "humanoid_wave_only.mp4"
print(f"Creating and downloading '{output_filename_mp4}'...")

with imageio.get_writer(output_filename_mp4, fps=30) as writer:
    for frame in frames:
        writer.append_data(frame)

files.download(output_filename_mp4)
print("--- Process Complete. ---")


Creating and downloading 'humanoid_wave_only.mp4'...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--- Process Complete. ---
